In [1]:
import sys
"""
This imports the sys module, which provides access to system-specific parameters and functions.
"""

from awsglue.transforms import *
"""
This imports all the classes and functions from the awsglue.transforms module. 
AWS Glue transforms are used for performing operations like mapping, filtering, aggregating, and joining data.
"""

from awsglue.utils import getResolvedOptions
"""
It is used to retrieve the resolved options from the command line arguments.
"""

from pyspark.context import SparkContext
"""
entry point for rdds
"""

from pyspark.sql import SparkSession
"""
Entry point in spark for using dataframe and sql functionalities

Difference between spark session and spark context:
Spark context is the entry point for using low level programming like RDD. It connects to a cluster.

"""
from pyspark.sql.functions import sum, count, when, col, min, row_number
"""
In this statement, we import various functions for transformations and aggregations.
"""

from awsglue.context import GlueContext
"""
GlueContext is a high-level interface for working with AWS Glue.
"""

from awsglue.job import Job
"""
Job represents an AWS Glue job.
"""

from pyspark.sql.window import Window
"""
Window is used to define specifications for window functions which operate on subset of rows.

In context of SQL:

Window functions applies aggregate and ranking functions over a particular window (set of rows). 
OVER clause is used with window functions to define that window. OVER clause does two things : 
    Partitions rows into form set of rows. (PARTITION BY clause is used) 
    Orders rows within those partitions into a particular order. (ORDER BY clause is used)

If partitions aren’t done, then ORDER BY orders all rows of table. 

Various aggregate functions such as SUM(), COUNT(), AVERAGE(), MAX(), MIN() applied 
over a particular window (set of rows) are called aggregate window functions. 
"""

from pyspark.sql import functions as F

In [2]:
args = getResolvedOptions(sys.argv, ["JOB_NAME"])
"""
retrieve the resolved options from the command line arguments. 
It looks for the value of the JOB_NAME argument and stores it in the args variable.
"""

sc = SparkContext()
"""
creating sparkContext object
"""

glueContext = GlueContext(sc)
"""
creating GlueContext object
"""

spark = glueContext.spark_session
"""
 This line retrieves the SparkSession associated with the glueContext. 
 The SparkSession is the entry point for working with structured data in Spark SQL
"""

job = Job(glueContext)
"""
This line creates a new Job object named job. 
The Job represents an AWS Glue job and provides methods for initializing and executing the job.
"""

job.init(args["JOB_NAME"], args)
"""
This line initializes the AWS Glue job by calling the init method of the Job object. 
It takes the JOB_NAME value from the args dictionary and initializes the job with that name. 
It also passes the args dictionary containing the resolved options to the init method.
"""

# Script generated for node Amazon S3
AmazonS3_node1686309256402 = glueContext.create_dynamic_frame.from_catalog(
    database="group-9-project-1-db",
    table_name="ipl_ball_by_ball_2008_2022_csv",
    transformation_ctx="AmazonS3_node1686309256402", 
    """
     This provides a unique name or identifier for this transformation context. 
     It is used internally by AWS Glue to track and manage transformations.
    """
)

"""
creating a dynamic frame from the data cataglog
"""


# Script generated for node S3 bucket
S3bucket_node1 = glueContext.create_dynamic_frame.from_catalog(
    database="group-9-project-1-db",
    table_name="ipl_matches_2008_2022_csv",
    transformation_ctx="S3bucket_node1",
)

# Script generated for node Change Schema
ChangeSchema_node1686309271115 = ApplyMapping.apply(
    frame=AmazonS3_node1686309256402,
    mappings=[
        ("id", "long", "id", "long"),
        ("innings", "long", "innings", "long"),
        ("overs", "long", "overs", "long"),
        ("ballnumber", "long", "ballnumber", "long"),
        ("batter", "string", "batter", "string"),
        ("bowler", "string", "bowler", "string"),
        ("non-striker", "string", "non-striker", "string"),
        ("extra_type", "string", "extra_type", "string"),
        ("batsman_run", "long", "batsman_run", "long"),
        ("extras_run", "long", "extras_run", "long"),
        ("total_run", "long", "total_run", "long"),
        ("non_boundary", "long", "non_boundary", "long"),
        ("iswicketdelivery", "long", "iswicketdelivery", "long"),
        ("player_out", "string", "player_out", "string"),
        ("kind", "string", "kind", "string"),
        ("fielders_involved", "string", "fielders_involved", "string"),
        ("battingteam", "string", "battingteam", "string"),
    ],
    transformation_ctx="ChangeSchema_node1686309271115",
)
"""
ApplyMapping.apply(...): This function applies mapping rules to the input dynamic frame to transform the data.
"""


# Script generated for node ApplyMapping
ApplyMapping_node2 = ApplyMapping.apply(
    frame=S3bucket_node1,
    mappings=[
        ("id", "long", "id", "long"),
        ("city", "string", "city", "string"),
        ("date", "string", "date", "string"),
        ("season", "string", "season", "string"),
        ("matchnumber", "string", "matchnumber", "string"),
        ("team1", "string", "team1", "string"),
        ("team2", "string", "team2", "string"),
        ("venue", "string", "venue", "string"),
        ("tosswinner", "string", "tosswinner", "string"),
        ("tossdecision", "string", "tossdecision", "string"),
        ("superover", "string", "superover", "string"),
        ("winningteam", "string", "winningteam", "string"),
        ("wonby", "string", "wonby", "string"),
        ("margin", "string", "margin", "string"),
        ("method", "string", "method", "string"),
        ("player_of_match", "string", "player_of_match", "string"),
        ("team1players", "string", "team1players", "string"),
        ("team2players", "string", "team2players", "string"),
        ("umpire1", "string", "umpire1", "string"),
        ("umpire2", "string", "umpire2", "string"),
    ],
    transformation_ctx="ApplyMapping_node2",
)

matches_df = ChangeSchema_node1686309271115.toDF()
ball_by_ball_df = ApplyMapping_node2.toDF()

"""
converting to dataframes
"""

In [7]:
joined_df = matches_df.join(ball_by_ball_df, on='ID')
"""
joining the two data frames using the join api and using key as "ID"
"""

joined_df = joined_df.filter(joined_df.extra_type != 'wides')
"""
filtering the rows to not include the rows where extra type is wides.
Wides are excluded as it is not counted as the valid delivery by the bowler. Thus it is not counted as ball faced by the batter.
"""

filtered_df = joined_df.select('Season', 'Date', 'batter', 'batsman_run', 'BattingTeam', 'Team1', 'Team2')
"""
dataset contains many columns, we do not require all of them, thus selecting relevant columns according to the requirement
"""
filtered_df.show()
"""
viewing the dataframe
"""

+------+----------+-----------+-----------+----------------+----------------+--------------+
|Season|      Date|     batter|batsman_run|     BattingTeam|           Team1|         Team2|
+------+----------+-----------+-----------+----------------+----------------+--------------+
|  2022|2022-05-29|YBK Jaiswal|          0|Rajasthan Royals|Rajasthan Royals|Gujarat Titans|
|  2022|2022-05-29|YBK Jaiswal|          0|Rajasthan Royals|Rajasthan Royals|Gujarat Titans|
|  2022|2022-05-29| JC Buttler|          1|Rajasthan Royals|Rajasthan Royals|Gujarat Titans|
|  2022|2022-05-29|YBK Jaiswal|          0|Rajasthan Royals|Rajasthan Royals|Gujarat Titans|
|  2022|2022-05-29|YBK Jaiswal|          0|Rajasthan Royals|Rajasthan Royals|Gujarat Titans|
|  2022|2022-05-29|YBK Jaiswal|          0|Rajasthan Royals|Rajasthan Royals|Gujarat Titans|
|  2022|2022-05-29| JC Buttler|          0|Rajasthan Royals|Rajasthan Royals|Gujarat Titans|
|  2022|2022-05-29| JC Buttler|          0|Rajasthan Royals|Rajasthan 

'\nviewing the dataframe\n'

In [ ]:
"""
this represents the ball by ball dataset
batter - at ball 0.1, the batter was YBK Jaiswal
batsman run - run scored by him at this ball was 0
batting team - RR
team 1 - RR
team 2 - GT
"""

In [8]:
# Add a ballnumber column using row_number()
window_spec = Window.partitionBy("Season", "Date", "batter").orderBy("batter")

"""
It partitions the data based on the columns "Season", "Date", and "batter" and 
orders the data within each partition by the "batter" column.

1) Partitioning according to the season
2) Partitioning according to the date because we want a particular match on which batter scores hundred.
3) Partioning by batter because we want individual batter's record
4) Ordering by batter to get a batter record sequentially on the same date and season
"""

result_df = filtered_df.withColumn("ballnumber", row_number().over(window_spec))
"""
generating row numbers based on the partition
for example: 
the dataset looks like this:

YBK Jaiswal
YBK Jaiswal
Jos Butler
Jos Butler
YBK Jaiswal
YBK Jaiswal

It is mixed. if we directly assingn a row number on it, it will look like this:

YBK Jaiswal 1
YBK Jaiswal 2
Jos Butler 3
Jos Butler 4
YBK Jaiswal 5
YBK Jaiswal 6

it is incorrect.


we want it like this:

YBK Jaiswal 1
YBK Jaiswal 2
YBK Jaiswal 3
YBK Jaiswal 4
Jos Butler 1
Jos Butler 2


In this way, we get the correct number of balls faced by the batter. It represents the ball number faced by a batter.
we are using window, because row number is a window function
"""

result_df.show()

+-------+----------+-----------+-----------+--------------------+--------------------+--------------------+----------+
| Season|      Date|     batter|batsman_run|         BattingTeam|               Team1|               Team2|ballnumber|
+-------+----------+-----------+-----------+--------------------+--------------------+--------------------+----------+
|2007/08|2008-04-18|  AA Noffke|          0|Royal Challengers...|Royal Challengers...|Kolkata Knight Ri...|         1|
|2007/08|2008-04-18|  AA Noffke|          1|Royal Challengers...|Royal Challengers...|Kolkata Knight Ri...|         2|
|2007/08|2008-04-18|  AA Noffke|          1|Royal Challengers...|Royal Challengers...|Kolkata Knight Ri...|         3|
|2007/08|2008-04-18|  AA Noffke|          4|Royal Challengers...|Royal Challengers...|Kolkata Knight Ri...|         4|
|2007/08|2008-04-18|  AA Noffke|          1|Royal Challengers...|Royal Challengers...|Kolkata Knight Ri...|         5|
|2007/08|2008-04-18|  AA Noffke|          0|Roya

In [9]:
# Calculate the cumulative sum of runs for each batter
window_spec = Window.partitionBy("Season","Date","batter").orderBy("ballnumber").rowsBetween(Window.unboundedPreceding, Window.currentRow)
"""
Defining another window specification to calculate runs

It partitions the data based on the columns "Season", "Date", and "batter", 
orders the data within each partition by the "ballnumber" column, and 
specifies the range of rows to include in the window. In this case, it includes all rows 
from the beginning of the partition (Window.unboundedPreceding) up to and including the current row (Window.currentRow).

"""

cumulative_df = result_df.withColumn("cumulative_runs", sum(col("batsman_run")).over(window_spec))

"""
calculates the cumulative sum of the "batsman_run" column within the window specified by window_spec. 
The sum function is applied to the column using the over method, which performs the aggregation over the specified window.

we are calculating cumulative runs to get point at which the batter will score hundred

"""

'\ncalculates the cumulative sum of the "batsman_run" column within the window specified by window_spec. \nThe sum function is applied to the column using the over method, which performs the aggregation over the specified window.\n\nwe are calculating cumulative runs to get point at which the batter will score hundred\n\n'

In [10]:
cumulative_df.show()

+-------+----------+-----------+-----------+--------------------+--------------------+--------------------+----------+---------------+
| Season|      Date|     batter|batsman_run|         BattingTeam|               Team1|               Team2|ballnumber|cumulative_runs|
+-------+----------+-----------+-----------+--------------------+--------------------+--------------------+----------+---------------+
|2007/08|2008-04-18|  AA Noffke|          0|Royal Challengers...|Royal Challengers...|Kolkata Knight Ri...|         1|            0.0|
|2007/08|2008-04-18|  AA Noffke|          1|Royal Challengers...|Royal Challengers...|Kolkata Knight Ri...|         2|            1.0|
|2007/08|2008-04-18|  AA Noffke|          1|Royal Challengers...|Royal Challengers...|Kolkata Knight Ri...|         3|            2.0|
|2007/08|2008-04-18|  AA Noffke|          4|Royal Challengers...|Royal Challengers...|Kolkata Knight Ri...|         4|            6.0|
|2007/08|2008-04-18|  AA Noffke|          1|Royal Chall

In [11]:
cumulative_df = cumulative_df.withColumn("4s", when(col("batsman_run") == 4, 1).otherwise(0))
"""
calculating number of fours by the batsman in a match.
if the batsman run is 4, then giving the value 1 to 4s column else giving 0
"""

cumulative_df = cumulative_df.withColumn("6s", when(col("batsman_run") == 6, 1).otherwise(0))
"""
same for six

withColumn api adds a new column to the dataframe. If column already exists, it gets overrided.
"""

cumulative_df = cumulative_df.withColumn("BF", when(col("cumulative_runs") >= 100, col("ballnumber")).otherwise(None))
"""
for this use case, the BF is equivalent to the point at which the batter reached 100. therefore we apply condition such that
when cumulative runs (represents the runs at a ballnumber) is greater than or equal to 100, 
then the ball faced will be equivalent to that ballnumber 
"""


'\nfor this use case, the BF is equivalent to the point at which the batter reached 100. therefore we apply condition such that\nwhen cumulative runs (represents the runs at a ballnumber) is greater than or equal to 100, \nthen the ball faced will be equivalent to that ballnumber \n'

In [12]:
cumulative_df.show()

+-------+----------+-----------+-----------+--------------------+--------------------+--------------------+----------+---------------+---+---+----+
| Season|      Date|     batter|batsman_run|         BattingTeam|               Team1|               Team2|ballnumber|cumulative_runs| 4s| 6s|  BF|
+-------+----------+-----------+-----------+--------------------+--------------------+--------------------+----------+---------------+---+---+----+
|2007/08|2008-04-18|  AA Noffke|          0|Royal Challengers...|Royal Challengers...|Kolkata Knight Ri...|         1|            0.0|  0|  0|null|
|2007/08|2008-04-18|  AA Noffke|          1|Royal Challengers...|Royal Challengers...|Kolkata Knight Ri...|         2|            1.0|  0|  0|null|
|2007/08|2008-04-18|  AA Noffke|          1|Royal Challengers...|Royal Challengers...|Kolkata Knight Ri...|         3|            2.0|  0|  0|null|
|2007/08|2008-04-18|  AA Noffke|          4|Royal Challengers...|Royal Challengers...|Kolkata Knight Ri...|     

In [15]:
cumulative_df = cumulative_df.withColumn("Against", when(col("BattingTeam") == col("Team1"), col("Team2")).otherwise(col("Team1")))
"""
to find the against team of the batter, we apply condition such that when batting team is equal to team1, then team2 is the against team
else team1 is the against team
"""



cumulative_df = cumulative_df.groupBy("Season","Date","batter","Against").agg(min(col("BF")).alias("BF"), \
                                                                     sum(col("4s")).alias("4s"), \
                                                                     sum(col("6s")).alias("6s"), \
                                                                     sum(col("batsman_run")).cast("integer").alias("Runs"))

"""
Aggregating to provide proper stastics 
"""

cumulative_df = cumulative_df.dropna()
"""
dropping rows with null valuse --> equivalent to saying that dropping all the records where a score of 100 or 
more is not scored by the batter
"""

cumulative_df = cumulative_df.withColumnRenamed("batter", "Player") \
                             .withColumnRenamed("Date", "Match Date") 

"""
renaming the columns
"""

cumulative_df = cumulative_df.orderBy("BF")
"""
ordering by BF as to get players who scored 100 or more with minimum balls
"""

result_df = cumulative_df.select("Season", "Player", "Runs", "BF", "4s", "6s", "Against", "Match Date")
"""
selecting columns according to requirements
"""

result_df.show()

+-------+--------------+----+---+---+---+--------------------+----------+
| Season|        Player|Runs| BF| 4s| 6s|             Against|Match Date|
+-------+--------------+----+---+---+---+--------------------+----------+
|   2013|      CH Gayle| 175| 30| 13| 17|       Pune Warriors|2013-04-23|
|2009/10|     YK Pathan| 100| 37|  9|  8|      Mumbai Indians|2010-03-13|
|   2013|     DA Miller| 101| 38|  8|  7|Royal Challengers...|2013-05-06|
|2007/08|  AC Gilchrist| 109| 42|  9| 10|      Mumbai Indians|2008-04-27|
|   2016|AB de Villiers| 129| 43| 10| 12|       Gujarat Lions|2016-05-14|
|   2017|     DA Warner| 126| 43| 10|  8|Kolkata Knight Ri...|2017-04-30|
|2007/08| ST Jayasuriya| 114| 45|  9| 11| Chennai Super Kings|2008-05-14|
|2020/21|    MA Agarwal| 106| 45| 10|  7|    Rajasthan Royals|2020-09-27|
|   2015|      CH Gayle| 117| 46|  7| 12|     Kings XI Punjab|2015-05-06|
|2009/10|       M Vijay| 127| 46|  8| 11|    Rajasthan Royals|2010-04-03|
|   2011|      CH Gayle| 107| 46| 10| 

In [ ]:
# Define the window specification to add a row number within each partition
window_spec = Window.partitionBy("Season").orderBy("BF")

# Add a new column starting with 1 within each partition
result_df = result_df.withColumn("POS", row_number().over(window_spec))

# Select the desired columns for the final result
result_df = result_df.select(
    "POS", "Season", "Player", "Runs", "BF", "4s", "6s", "Against", "Venue", "Match Date"
)

# Define the output path for storing the partitioned data
output_path = "s3://aws-group-9/Project-1/results/Requirement-1-Fastest-Century/"

# Write the partitioned data to the output path as a single combined file in each partition folder
result_df.coalesce(1).write.partitionBy("Season").csv(output_path, header=True, mode="overwrite")

"""
coalesce(1): This operation reduces the number of output files to a single file by merging the partitions of the DataFrame into one partition. 
The number 1 specifies the target number of partitions.

If you use coalesce(0) instead of coalesce(1), it will attempt to reduce the number of partitions to zero. 
However, setting the number of partitions to zero is not valid and will result in an error.

If you attempt to use coalesce(0), you will encounter an exception such as IllegalArgumentException: requirement failed: 
The number of partitions must be positive. 
This is because Spark requires at least one partition to distribute and process the data.
Therefore, it is not possible to set the number of partitions to zero using the coalesce method.

"""

job.commit()
"""
The job.commit() statement is used to explicitly commit a job in AWS Glue. 
In AWS Glue, a job represents an ETL (Extract, Transform, Load) operation that processes and transforms data. 
The commit() method is called to indicate the successful completion of the job.
"""
